In [1]:
%matplotlib inline
from __future__ import division
from os.path import join, isfile
import pandas as pd
import numpy as np

conditions = ['Physics', 'NonPhysics']
tr = 2.
n_vols = 167
cutoff = .3333333333333333333333
hrf = np.array([0, 0.0866, 0.3749, 0.3849, 0.2161, 0.0769, 
                0.0016, -0.0306, -0.0373, -0.0308, -0.0205, 
               -0.0116, -0.0058, -0.0026, -0.0011, -0.0004, -0.0001])

sum_f = '/home/data/nbc/physics-learning/physics-learning/fci-motion-censor-summary.txt'
sum_df = pd.read_csv(sum_f, sep='\t')
sum_df = pd.melt(sum_df, value_vars=['fci-0', 'fci-1', 'fci-2'],
                 value_name='prop_bad',
                 id_vars=['subject', 'session'], var_name='run')
for sub_num in sum_df['subject'].unique():
    sub = str(sub_num)
    sub_df = sum_df.loc[sum_df['subject']==sub_num]
    for ss in sub_df['session'].unique():
        sess = 'session-{0}'.format(int(ss))
        sess_df = sub_df.loc[sub_df['session']==ss]
        for run in sess_df['run'].unique():
            fold = join('/home/data/nbc/physics-learning/data/behavioral-data/vectors',
                        sub, sess, 'fci/')
            fold2 = join('/home/data/nbc/physics-learning/data/first-level',
                         sub, sess, 'fci/', run)
            f2 = join(fold2, '{0}-{1}-motion-outliers-censored.txt'.format(sub, sess))
            if isfile(f2):
                censor_data = np.loadtxt(f2)
                if censor_data.shape == 2:
                    censor_data = np.sum(censor_data, axis=1)
                censor_vols = np.where(censor_data)[0]

                for c in conditions:
                    f = join(fold, '{0}-{1}.txt'.format(run, c))
                    df = pd.read_csv(f, sep='\t', names=['onset', 'duration', 'amplitude'])
                    temp_dat = np.zeros(n_vols)

                    # convert vectors from secs to trs
                    df['onset'] = np.round(df['onset'] / tr).astype(int)
                    df['duration'] = np.round(df['duration'] / tr).astype(int)

                    for i in range(df.shape[0]):
                        s = df.loc[i]['onset']
                        e = s + df.iloc[i]['duration']
                        temp_dat[s:e] = 1

                    pred_dat = np.convolve(temp_dat, hrf)[:n_vols]
                    cond_vols = np.where(pred_dat!=0)[0]
                    inter = np.intersect1d(cond_vols, censor_vols)
                    prop_bad = len(inter) / len(cond_vols)
                    if prop_bad >= cutoff:
                        print('{0} {1} {2} {3}: {4}'.format(sub, sess, run, c, prop_bad))
            else:
                pass
#                for c in conditions:
                    # This shouldn't be here. If the censored vols file doesn't
                    # exist, it should skip that run, since there are no
                    # outliers. Currently, it is comparing the prop_bad from
                    # the *last* run to the cutoff
#                    if prop_bad >= cutoff:
#                        print('{0} {1} {2} {3}: 0'.format(sub, sess, run, c))

/home/data/nbc/data-analysis/env/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


102 session-0 fci-1 NonPhysics: 0.333333333333
102 session-1 fci-1 Physics: 0.515151515152
102 session-1 fci-1 NonPhysics: 0.515151515152
102 session-1 fci-2 Physics: 0.369863013699
571 session-1 fci-1 NonPhysics: 0.405797101449
597 session-0 fci-2 NonPhysics: 0.361111111111
611 session-0 fci-2 NonPhysics: 0.397058823529
622 session-0 fci-2 Physics: 0.402777777778
622 session-0 fci-2 NonPhysics: 0.354838709677
